In [5]:
import os
#Set current location to the location of the script
os.chdir("/dust3r")
print(os.getcwd())

/dust3r


In [6]:
#Display imgs
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import open3d as o3d
import torch

from dust3r.inference import inference, inference_with_mask
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

DATA_PATH = "/dust3r/masked_dust3r/data/jackal_irl_one_spin"
device = 'cuda'
batch_size = 1
schedule = 'cosine'
lr = 0.01
niter = 300


In [7]:
model_name = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
# you can put the path to a local checkpoint in model_name if needed
model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)

... loading model from checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth


instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>


In [8]:
# load_images can take a list of images or a directory

images_array = []
masks_array = []

for i in range(0,20,2):
    images_array.append(os.path.join(DATA_PATH,"masked_images/{}.png".format(i)))
    masks_array.append(os.path.join(DATA_PATH,"masks/{}.png.png".format(i)))
images = load_images(images_array, size=512, verbose=True)

masks = []

for i in range(len(masks_array)):
    #Open as mask and load to gpu
    mask = Image.open(masks_array[i]).convert('L')
    #Resize to match image size
    _,_,H,W = images[i]["img"].shape
    mask = mask.resize((W,H))

    mask = np.array(mask)
    mask = torch.tensor(mask).to(device)/255
    masks.append(mask)

>> Loading a list of 10 images
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/0.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/2.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/4.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/6.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/8.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/10.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/12.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/masked_images/14.png with resolution 640x480 --> 512x384
 - adding /dust3r/masked_dust3r/data/jackal_irl_one_spin/maske

In [9]:
pairs = make_pairs(images, scene_graph='complete', prefilter=None, symmetrize=True)
output = inference_with_mask(pairs, model, device, masks, batch_size=batch_size)
#output = inference(pairs, model, device, batch_size=batch_size)

view1, pred1 = output['view1'], output['pred1']
view2, pred2 = output['view2'], output['pred2']

>> Inference with model on 90 image pairs


  0%|          | 0/90 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 90/90 [00:47<00:00,  1.89it/s]


In [10]:
scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PointCloudOptimizer)
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()

 init edge (6*,3*) score=1.0942810773849487
 init edge (6,7*) score=1.0832068920135498
 init edge (6,8*) score=1.0725075006484985
 init edge (6,5*) score=1.046488881111145
 init edge (6,9*) score=1.043563961982727
 init edge (6,4*) score=1.0320390462875366
 init edge (6,2*) score=1.023155689239502
 init edge (3,1*) score=1.0219547748565674
 init edge (3,0*) score=1.011500358581543
 init loss = 0.00011270594404777512
Global alignement - optimizing for:
['pw_poses', 'im_depthmaps', 'im_poses', 'im_focals']


100%|██████████| 300/300 [01:01<00:00,  4.85it/s, lr=1.27413e-06 loss=2.30851e-05]


In [11]:
#Check if pointclouds folder exists
#If exists, delete all files in the folder
if os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    for file in os.listdir("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
        os.remove("{DATA_PATH}/pointclouds/{file}".format(DATA_PATH=DATA_PATH, file=file))
        
if not os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    os.makedirs("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH))

for i in range(len(images)):
    pointcloud = pts3d[i].detach().cpu().numpy()
    pointcloud = pointcloud.reshape(-1, 3)
    color = imgs[i].reshape(-1, 3)
    confidence_mask = confidence_masks[i].detach().cpu().numpy()
    confidence_mask = confidence_mask.reshape(-1)
    
    masked_pointcloud = []
    masked_color = []

    for j in range(len(confidence_mask)):
        if confidence_mask[j]:
            masked_pointcloud.append(pointcloud[j])
            masked_color.append(color[j])

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(masked_pointcloud)
    pcd.colors = o3d.utility.Vector3dVector(masked_color)
    o3d.io.write_point_cloud("{DATA_PATH}/pointclouds/pointcloud{i}.ply".format(DATA_PATH=DATA_PATH, i=i), pcd)

[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.


(-0.5, 511.5, 383.5, -0.5)

In [12]:
conf1 = confidence_masks[2].cpu().numpy()
conf2 = confidence_masks[9].cpu().numpy()

plt.subplot(1, 2, 1)
plt.imshow(conf1, cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(conf2, cmap='gray')
plt.axis('off')